In [13]:
"""
Evaluate a CNN classifier on holdout test set.
Copyright (c) 2023 Global Health Labs, Inc
The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.
"""

import os
import time
import sys
import glob
import random
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
from CNN.data_generator import DataGeneratorMemory,DataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score, precision_score, recall_score, confusion_matrix,f1_score

In [4]:
##### Configurations #####
val_pos_dir='/workspace/data/EndorTeamArea/skulhare/lrs_ped_con/frames/holdout/consolidation' # directory with positive images
val_neg_dir='/workspace/data/EndorTeamArea/skulhare/lrs_ped_con/frames/holdout/no_consolidation' # directory with negative images
os.environ["CUDA_VISIBLE_DEVICES"] = "1" # it will occupy GPU randomly if it's not limited by the system
IMG_SHAPE = (256, 256, 1)
batch_size = 1
experiment_dir = 'logs/75/855/combine'

In [5]:
##### Data Setup #####
validation_images_1 = glob.glob(val_pos_dir + '/*.jpg')+glob.glob(val_pos_dir + '/**/*.jpg')
validation_images_2 = glob.glob(val_neg_dir + '/*.jpg')+glob.glob(val_neg_dir + '/**/*.jpg')
validation_images=validation_images_1+validation_images_2

random.shuffle(validation_images)
validation_labels = [1 if val_pos_dir in image else 0 for image in validation_images]

validation_datagen = DataGeneratorMemory(validation_images,
                                    validation_labels,
                                    IMG_SHAPE,
                                    batch_size=batch_size,
                                    n_classes=2,
                                    shuffle=True,
                                    balanced=False)

In [6]:
def find_best(basedir):
    """ find the best CNN classifier with the highest validation accuracy
    Args:
      basedir:  a folder path which follows the underlining structure (default results of CNN classifier)
     └──basedir
          ├── modelA      (hyper-parameter setting A)
          │   ├── saved-model-01-0.xxxx.hdf5
          │   ├── saved-model-02-0.yyyy.hdf5
          :   :
          └── modelB     (hyper-parameter setting A)
              ├── saved-model-01-0.zzzz.hdf5
              :
              └── saved-model-100-0.aaaa.hdf5
    :return: model checkpoint path with the highest validation accuracy
    """
    model_list = glob.glob(basedir +'/**/*.hdf5')
    maxi = 0.0 
    best = None
    for model in model_list:
        acc = float(model.split('.')[-2].split('-')[-1])
        if acc>maxi:
            maxi = acc
            best = model
    return best

In [7]:
model_path = find_best(experiment_dir)

In [8]:
model = keras.models.load_model(model_path)
        
# if we want to train with other values:
model.training = False # to shut down the drop out and batch normalization
scores = model.predict_generator(validation_datagen)

# Convert predictions to binary values (0 or 1)
y_pred_binary = [1 if pred[1] > 0.5 else 0 for pred in scores]
consolidation_score = [score[1] for score in scores]

2023-08-24 23:36:09.521818: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-08-24 23:36:09.570822: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1082] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-24 23:36:09.571014: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1666] Found device 0 with properties: 
name: NVIDIA RTX A6000 major: 8 minor: 6 memoryClockRate(GHz): 1.8
pciBusID: 0000:4b:00.0
2023-08-24 23:36:09.571264: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2023-08-24 23:36:09.586343: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2023-08-24 23:36:09.620620: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcufft.so.

In [11]:
# Compute ROC and AUROC
fpr, tpr, thresholds = roc_curve(validation_datagen.labels, consolidation_score)
roc_auc = roc_auc_score(validation_datagen.labels, consolidation_score)

# Compute accuracy, and f1
accuracy = accuracy_score(validation_datagen.labels, y_pred_binary)
f1 = f1_score(validation_datagen.labels, y_pred_binary, average='binary')

# Print the evaluation metrics
print("ROC AUC:", roc_auc)
print("Accuracy:", accuracy)
print("F1 Score:", f1)

ROC AUC: 0.9693253441938101
Accuracy: 0.9299134612220723
F1 Score: 0.8953865295831832
